In [1]:
%matplotlib inline

import sys
sys.path.append("../")
from utils import DetectionDataset, draw_rectangle
from models.generator import DetectionGenerator, PriorBoxes
from models.ssd import build_base_network, attach_multibox_head
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt

# \[ 모델 학습 \]
----
----

우리는 학습할 모델과 데이터를 이전 시간에서 꾸렸습니다. 

> 이제 학습까지 남은 것은 `LOSS` 함수를 어떻게 설계할까? 입니다.

## 1. Loss 계산하기
----

원문에 있는 설명을 가져왔습니다. 우리가 학습해야 하는 것은 위치를 추론하는 Regressor와 사물을 분류하는 Classifer입니다. Regressor의 경우에는 SmoothL1이라 불리는 Loss로 학습을 시키고, Classifier은 분류모델에서 주로 사용하는 Cross-Entropy Loss를 이용합니다.<br>
Regressor의 경우에는 당연하게도 Matched prior box의 경우 한에서만 학습을 해야합니다.

The SSD Training Objective is derived from the MultiBox Objective, but is extended to handle multiple object categories. Let $x^p_{ij}= {1,0}$ be an indicator for matching the i-th default box to the j-th ground truth box of category $p$. In the matching strategy above, we can have $\sum_i x^{p}_{ij} \ge 1$. The overall objective loss function is a weighted sum of the localization loss(loc) and the confidence loss(conf):

$
L(x,c,l,g)= \frac{1}{N}(L_{conf}(x,c)+\alpha L_{loc}(x,l,g)) 
$

Where N is the number of matched default boxes. if $N = 0$, we set the loss to 0. The localization loss is a Smooth L1 loss between the predicted box(l) and the ground truth box(g) parameters. Similar to Faster R-CNN, we regress to offsets for the center$(cx,cy)$ of the default bounding box(p) and for its width(w) and height (h).<br>

$
L_{loc}(x,l,g) = \sum^{N}_{i\in Pos} \sum_{m \in {cx,cy,w,h}} x_{ij}^k smooth_{L1}(l^m_i-\hat g^m_j) \\
\hat g^{cx} = \frac{(g^{cx} - p^{cx})}{p^{w}}, \hat g^{cy} = \frac{(g^{cy} - p^{cy})}{p^{h}}\\  
\hat g^{w} = log(\frac{g^{w}}{p^{w}}), \hat g^{h} = log(\frac{g^{h}}{p^{h}}) \\
$<br>
The confidence loss is the softmax loss over multiple classes confidences (c).<br>
$
L_{conf}(x,c) = - \sum^{N}_{i\in Pos} x^{P}_{ij}log(\hat c^p_i) - \sum^{N}_{i\in Neg} log(\hat c_i^p) where \hat c^p_i = \frac{exp(c_i^p)}{\sum_p exp(c^p_i)}
$<br>

Confidence Loss를 계산할 때, 중요한 문제가 하나 있습니다. 바로 Class Imbalance 문제입니다. 영상에서 대부분은 BackGround에 해당합니다. 우리가 원하는 Foreground에 매칭된 Prior box는 극히 일부분에 불과합니다. 이 때문에, Easy Negative Sample, 즉 Background에 대한 Loss가 지나치게 커서, 실제로 학습하고자 하는 Foreground에 대한 학습은 잘 이루어지지 않게 됩니다. 이를 방지하기 위해, Negative Sample 중 가장 Loss가 큰것들을 위주로만 추출하여 학습하도록 합니다.

In [13]:
def SSDLoss(alpha=1., pos_neg_ratio=3.):
    def ssd_loss(y_true, y_pred):
        num_classes = tf.shape(y_true)[2] - 4
        y_true = tf.reshape(y_true, [-1, num_classes + 4])
        y_pred = tf.reshape(y_pred, [-1, num_classes + 4])
        eps = K.epsilon()

        # Split Classification and Localization output
        y_true_clf, y_true_loc = tf.split(y_true, 
                                          [num_classes, 4], 
                                          axis=-1)
        y_pred_clf, y_pred_loc = tf.split(y_pred, 
                                          [num_classes, 4], 
                                          axis=-1)

        # split foreground & background
        neg_mask = y_true_clf[:, -1]
        pos_mask = 1 - neg_mask
        num_pos = tf.reduce_sum(pos_mask)
        num_neg = tf.reduce_sum(neg_mask)
        num_neg = tf.minimum(pos_neg_ratio * num_pos, num_neg)

        # softmax loss
        y_pred_clf = K.clip(y_pred_clf, eps, 1. - eps)
        clf_loss = -tf.reduce_sum(y_true_clf * tf.log(y_pred_clf),
                                  axis=-1)
        pos_clf_loss = tf.reduce_sum(clf_loss * pos_mask) / (num_pos + eps)
        neg_clf_loss = clf_loss * neg_mask
        values, indices = tf.nn.top_k(neg_clf_loss,
                                      k=tf.cast(num_neg, tf.int32))
        neg_clf_loss = tf.reduce_sum(values) / (num_neg + eps)
        clf_loss = pos_clf_loss + neg_clf_loss
        
        # smooth l1 loss
        l1_loss = tf.abs(y_true_loc - y_pred_loc)
        l2_loss = 0.5 * (y_true_loc - y_pred_loc) ** 2
        loc_loss = tf.where(tf.less(l1_loss, 1.0),
                            l2_loss,
                            l1_loss - 0.5)
        loc_loss = tf.reduce_sum(loc_loss, axis=-1)
        loc_loss = tf.reduce_sum(loc_loss * pos_mask) / (num_pos + eps)

        # total loss
        return clf_loss + alpha * loc_loss
    return ssd_loss

## 2. 모델 구성하기
---


In [14]:
base_network = build_base_network(num_units=16)
predictions = attach_multibox_head(base_network,
                                   ['norm3_2','norm4_2','norm5_2'])
model = Model(base_network.input, predictions,name='ssd')

## 3. 데이터 구성하기
---


In [15]:
trainset = DetectionDataset(data_type='train')

prior = PriorBoxes(strides=[4, 8, 16],
           scales=[10, 25, 40],
           ratios=[0.4,0.8, 1.2])

traingen = DetectionGenerator(trainset, prior)

In [16]:
model.compile('adam',loss=ssd_loss)

In [17]:
model.fit_generator(traingen)

248/343 [====================>.........] - ETA: 3:50 - loss: 7.4274

KeyboardInterrupt: 